# Project Benson

## Aim

- High Gala Participation Rate

## How?

- Identifying High Traffic Volumes at Subway Entrances/Exits
 - By Day of Week
 - By Hour of Day



- Identifying Target Audience (those more likely to be interested)
 - Identifying Stations with High Volumes of Non-Tourist Traffic (locals)
     - By comparing station's weekend vs weekday traffic

 - [Demographic & Geographic Profiles](https://datausa.io/profile/geo/new-york-ny)
     - Assume areas with more females more interested?
     - Assume areas with higher income more likely to donate?
     - Assume areas with younger demographics more interested?
     - Assume areas with voters who support liberal party more interested?
     - Assume areas with higher education level more interested?
     - Assume areas with high employment in tech industry more interested?
  